Генераторы производят значения, а корутины потребляют

In [8]:

def grep(pattern):
    print("start grep")
    while True:
        # Функция замораживается и ждёт ввода данных
        # при помощи метода send 
        line = yield
        if pattern in line:
            print(line)


# Создание корутины
g = grep("python")
# Запуск корутины. Запускается код функции до 
# инструкции yield, после чего возвращает 
# управление в основной поток 
next(g)
# Отправляет данные в функцию в переменную line,
# т к именно там прописана конструкция yield
g.send("Golang is better")
g.send("python is sumple!")

# В переменной g зписано состояние функции

start grep
python is sumple!


In [9]:
def grep(pattern):
    print("start grep")
    try:
        while True:
            line = yield
            if pattern in line:
                print(line)
    # Обработка закрытия 
    except GeneratorExit:
        print("stop grep")
        
g = grep("python") 
next(g)
g.send("Golang is better")
g.send("python is sumple!")
# Закрытие корутины(руками), close может быть вызван сборщиком мусора
# Исключение генерирутеся в месте, где функция заморозила свою работу 
g.close()

start grep
python is sumple!
stop grep


In [10]:
def grep(pattern):
    print("start grep")
    try:
        while True:
            line = yield
            if pattern in line:
                print(line)
    # Обработка закрытия 
    except GeneratorExit:
        print("stop grep")
        
g = grep("python") 
next(g)
g.send("Golang is better")
# Отправка исключения в корутину для её завершения. Нужно отдельно обрабатывать.
# Исключение генерирутеся в месте, где функция заморозила свою работу 
g.throw(RuntimeError, "something wrong")

start grep


RuntimeError: something wrong

In [ ]:
# PEP 380 - стандарт для вызова из одних корутин другие
def grep(pattern):
    print("start grep")
    while True:
        line = yield
        if pattern in line:
            print(line)

def grep_python_corutine():
    g = grep("python")
    next(g)
    g.send("python is the best")
    g.close()
    
# g не является корутиной. Это обычная функция
g = grep_python_corutine()  

In [14]:
# Пример PEP 380. Делегирование вызова функции другой
def grep(pattern):
    print("start grep")
    while True:
        line = yield
        if pattern in line:
            print(line)

# Вызов корутины корутиной
def grep_python_corutine():
    g = grep("python")
    yield from g
    
g = grep_python_corutine()
print(g)
# "Запуск" корутины. Обязательно именно send(None)
g.send(None)
g.send("python wow!")

<generator object grep_python_corutine at 0x0000021F10F3E890>
start grep
python wow!


In [17]:
# В функцию передаём итерабельный объект
def chain(x_iter, y_iter):
    yield from x_iter
    yield from y_iter
    
# Аналогично chain
def the_same_chain(x_iter, y_iter):
    for x in x_iter:
        yield x
        
    for y in y_iter:
        yield y

        
a = [1, 2, 3]
b = (4, 5)
for x in chain(a, b):
    print(x)

1
2
3
4
5
